In [1]:
import concurrent.futures
import importlib
import openai
import os
import requests
import bs4
import traceback
from bs4 import BeautifulSoup
from joblib import Memory
from itertools import islice
openai.organization = "org-RfxFQjm7zizJjdQRALbJZaZB"
openai.api_key = os.environ.get("OPENAI_API_KEY")
modelToUse = "gpt-3.5-turbo"
print(modelToUse)
memory = Memory("/app/cachedata", verbose=0)


import scripts.marbot as marbot

gpt-3.5-turbo


In [2]:
importlib.reload(marbot) # little hack since I jump back and forth
marbot.loadUrls()

Fetching https://www.deskera.com/blog/paper-manufacturing-critical-issues-and-challenges/
Saved 104043 chars to /app/testdata/Paper-0.html
Fetching https://scitechconnect.elsevier.com/challenges-and-opportunities-for-the-pulp-and-paper/
ERROR: Failed to fetch https://scitechconnect.elsevier.com/challenges-and-opportunities-for-the-pulp-and-paper/
Fetching https://www.hammondpaper.com/blog/post/challenges-and-opportunities-for-the-paper-and-pulp-industry
Saved 32750 chars to /app/testdata/Paper-2.html
Fetching https://www.piworld.com/article/addressing-the-challenges-in-the-paper-supply-chain/
Saved 144049 chars to /app/testdata/Paper-3.html
Fetching https://www.youris.com/energy/renewables/refusing-to-fold-can-paper-manufacturers-face-down-energy-challenges-.kl
Saved 37717 chars to /app/testdata/Paper-4.html
Fetching https://creative-solution.com/global-paper-shortage-challenges-to-hit-direct-marketers-in-2022/
Saved 45993 chars to /app/testdata/Paper-5.html
Fetching https://linchpinse

In [3]:
importlib.reload(marbot) # little hack since I jump back and forth
filePaths = marbot.scanFiles("/app/testdata")
print(f"Found {len(filePaths)} industries with {sum([len(filePaths[filePath]) for filePath in filePaths])} pages")

Found 15 industries with 140 pages


In [5]:
importlib.reload(marbot) # little hack since I jump back and forth
import traceback ## remove in a little while
industryPageSummaries = {}
industrySummaries = {}
failedPages = []
for industry in sorted(filePaths):
    thisIndustryFilePaths = filePaths[industry]
    pageSummaries = []
    for filePath in thisIndustryFilePaths:
        try:
            pageSummary = marbot.loadAndSummarizePage(filePath, modelToUse)
            pageSummaries.append(pageSummary)
        except Exception as e:
            errorMessage = f"Error on {filePath}: {e}\n{traceback.format_exc()}"
            print(errorMessage)
            failedPages.append(errorMessage)
    industryPageSummaries[industry] = pageSummaries

if len(failedPages) > 0:
    print(f"Failed to summarize {len(failedPages)} pages")
    print("\n\n".join(failedPages))

Loading /app/testdata/Aircraft-4.html
Read 69682 chars from /app/testdata/Aircraft-4.html
Read 15651 chars of human readable text
Split into 4 batches of max 4000 chars
Submitting batch 0
Submitting batch 1
Submitting batch 2
Submitting batch 3
Summary 0: The North America Aerospace Parts Manufacturing Market is projected to experience a CAGR growth of 2.3% by 2030, reaching a market size of $504.46 billion USD. The industry focuses on producing durable, lightweight, and resistant components for aircraft and spacecraft. The manufacturing of aerospace parts requires stringent precision tolerances. The industry is driven by the need to meet the demands of passengers for safer and better traveling experiences, leading to the development of new accessories and services. Overall, the aerospace parts manufacturing market in North America is expected to continue growing in the coming years.

Major themes in the text:
- CAGR growth projection of 2.3% by 2030
- Focus on producing durable and li

In [49]:
for industry in sorted(filePaths):
    print(f"Num and length of page summaries for {industry}: {len(industryPageSummaries[industry])}: {list(len(x) for x in industryPageSummaries[industry])}")
    

Num and length of page summaries for Aircraft: 12: [1652, 1474, 1823, 3072, 1558, 2003, 733, 2228, 2088, 1518, 1730, 1006]
Num and length of page summaries for Appliance: 7: [1957, 1757, 1974, 216, 1062, 1632, 502]
Num and length of page summaries for Bakerybread: 11: [973, 2118, 358, 359, 1288, 1402, 114, 1118, 933, 2213, 1629]
Num and length of page summaries for Chemical: 8: [205, 1524, 2256, 2558, 1921, 2036, 1729, 1838]
Num and length of page summaries for Dairy: 8: [1374, 659, 1398, 1400, 1308, 1036, 333, 2145]
Num and length of page summaries for Electrical: 12: [1414, 1643, 1422, 1854, 1870, 1716, 2323, 2723, 2905, 2027, 1078, 1671]
Num and length of page summaries for Equipment: 2: [2412, 2003]
Num and length of page summaries for Equipment Manufacturing: 2: [1312, 971]
Num and length of page summaries for Food: 15: [1785, 1961, 743, 309, 5186, 2368, 1485, 827, 524, 2239, 403, 1133, 1103, 2991, 1854]
Num and length of page summaries for Motorvehicleparts: 11: [1689, 802, 1578,

In [50]:
importlib.reload(marbot) # little hack since I jump back and forth
industrySummaries = {}
for industry in sorted(industryPageSummaries):
    print(f"Combining {industry}")
    pageSummaries = industryPageSummaries[industry]
    combinedSummary = marbot.combinePageSummaries(pageSummaries, "gpt-4")
    industrySummaries[industry] = combinedSummary
    print(f"Length of overall summary: {len(industrySummaries[industry])}")
    print(f"Overall summary:\n{industrySummaries[industry]}")

Combining Aircraft


Combined page summaries:
"COVID-19 Impact" (8): The aerospace industry faced significant disruption and production halts due to COVID-19. While long-term effects remain uncertain, recovery has been increasingly evident, particularly driven by defense manufacturing and expectations of growing travel demand.

"Digital Transformation" (4): Companies in the aerospace industry are tapping into digital technologies, such as dynamic data layers, to improve operations. Blockchain technology and smart factories are among the emerging trends for increasing efficiency, productivity, and data-driven decision-making.

"Supply Chain Disruptions" (4): Challenges such as disruptions in the supply chain, parts shortages, and increasing costs have been experienced across the industry. Advanced analytics and a tailored approach to supply chain management are seen as potential solutions.

"Rising Costs" (4): The rising manufacturing and operational costs are a key challenge in the ind